## Identifying relevant new origins of replication


In [1]:
using DataFrames

In [90]:
## This opens the count table generated by the original Unique.seqs output and creates 
# a sorted data frame mapping sequence name to frequency

io = open("Unique.seqs.count_table", "r")
count_lines = readlines(io)
close(io)

frequency_table = DataFrame(sequence_name = String[], frequency = Int64[])

array = [split(line, "\t") for line in count_lines]
for map = 2 : size(array,1)
    push!(frequency_table, [array[map][1], parse(Int64, array[map][2])])
end

sort!(frequency_table, [:frequency], rev = true);

In [91]:
## This opens the fasta list of sequences generated by the original Unique.seqs output
# it creates a second data frame matching name to sequence

io = open("Unique.seqs.fasta", "r")
seq_lines = readlines(io)
close(io)

sequences = DataFrame(seq_name = String[], DNA_sequence = String[])

state = "keys"
seq_name = ""
DNA_sequence = ""
for line in seq_lines
    if line[begin] == '>' && state == "keys"
        seq_name = split(line, '>')[2]
        state = "info"
    elseif state == "info"
        DNA_sequence = line
        state = "keys"
    end
    if seq_name != "" && DNA_sequence != ""
        push!(sequences, [seq_name, DNA_sequence])
        seq_name = ""
        DNA_sequence = ""
    end
end

sequences;

In [92]:
## This routine creates a new DataFrame, combining the earlier two made to identify the most common sequences

min_freq = 5
final_table = DataFrame(seq_name = String[], frequency = Int64[], DNA_sequence = String[])

for scan = 1 : size(frequency_table, 1)
    if frequency_table[scan,2] > min_freq
        DNA_sequence = sequences[in([frequency_table[scan,1]]).(sequences.seq_name), :].DNA_sequence[1]
        seq_name = ">" * frequency_table[scan,1]
        push!(final_table, [seq_name, frequency_table[scan,2], DNA_sequence])
    end
end

final_table;

In [93]:
## This routine parses the table back to FASTA format

io = open("Most_common_origins.fa", "w")

for event = 1 : size(final_table, 1)
    write(io, final_table[event,1][1:8]*final_table[event,1][35:end]* "_freq:" * string(final_table[event,2]) * '\n')
    write(io, final_table[event,3] * '\n')
end

close(io)